In [1]:
import warnings

In [2]:
warnings.filterwarnings("ignore")

In [3]:
import pandas
from sklearn.model_selection import train_test_split
from keras import Sequential
from keras.layers import Embedding, Dense, Dropout, GlobalMaxPool1D, Flatten
from keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import tensorflow
import numpy

Using TensorFlow backend.


In [4]:
o = pandas.read_csv('small-preprocessed_onion_data-500-rows.csv')

In [5]:
X = o.iloc[:, :-1]

In [6]:
Y = o.iloc[:, -1]

In [7]:
x_fit, x_test, y_fit, y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

In [8]:
x_train, x_val, y_train, y_val = train_test_split(x_fit, y_fit, test_size=0.20, random_state=42)

In [9]:
x_train = tensorflow.keras.utils.normalize(x_train, axis=1).to_numpy()
x_val = tensorflow.keras.utils.normalize(x_val, axis=1).to_numpy()
x_test = tensorflow.keras.utils.normalize(x_test, axis=1).to_numpy()

In [10]:
y_train = y_train.to_numpy()
y_val = y_val.to_numpy()
y_test = y_test.to_numpy()

In [11]:
model = Sequential()

model.add(Dense(128, activation=tensorflow.nn.relu))
model.add(Dense(128, activation=tensorflow.nn.relu))
model.add(Dense(2, activation=tensorflow.nn.softmax))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [12]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)

In [13]:
model.fit(x_train, y_train, epochs=10, batch_size=16, verbose=1, validation_data=(x_val, y_val), callbacks=[es])

Train on 300 samples, validate on 75 samples
Epoch 1/10
300/300 [==============================] - 0s 675us/step - loss: 0.6813 - accuracy: 0.6067 - val_loss: 0.6529 - val_accuracy: 0.6400
Epoch 2/10
300/300 [==============================] - 0s 299us/step - loss: 0.6010 - accuracy: 0.6067 - val_loss: 0.6087 - val_accuracy: 0.6400
Epoch 3/10
300/300 [==============================] - 0s 286us/step - loss: 0.3998 - accuracy: 0.8767 - val_loss: 0.4909 - val_accuracy: 0.8267
Epoch 4/10
300/300 [==============================] - 0s 278us/step - loss: 0.1221 - accuracy: 0.9933 - val_loss: 0.3647 - val_accuracy: 0.8533
Epoch 5/10
300/300 [==============================] - 0s 278us/step - loss: 0.0185 - accuracy: 1.0000 - val_loss: 0.3446 - val_accuracy: 0.8667
Epoch 6/10
300/300 [==============================] - 0s 301us/step - loss: 0.0050 - accuracy: 1.0000 - val_loss: 0.3411 - val_accuracy: 0.8800
Epoch 7/10
300/300 [==============================] - 0s 296us/step - loss: 0.0025 - accura

In [14]:
y_pred = model.predict(x_test)

In [15]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               337280    
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 258       
Total params: 354,050
Trainable params: 354,050
Non-trainable params: 0
_________________________________________________________________


In [16]:
final_evaluation = []
y_hat = []
y_real = []
evaluation = []

In [17]:
y_test_hat = [y_test, numpy.zeros(y_test.shape[0], dtype=numpy.int16)]

In [18]:
for i in range(len(y_test)):
    if y_test_hat[0][i] == 0:
        y_test_hat[1][i] = 1
    else:
        y_test_hat[1][i] = 0

In [19]:
y_test_hat = numpy.transpose(y_test_hat)

In [20]:
y_test_hat = y_test_hat[:,[0, 1]] = y_test_hat[:,[1, 0]]

In [21]:
for i in range(len(y_test)):
    final_evaluation.append(y_test_hat[i][0] - y_pred[i][0])
    y_real.append(y_test_hat[i][0])
    y_hat.append(y_pred[i][0])
    if numpy.mean(numpy.abs(y_test_hat[i] - y_pred[i])) > 0.5:
        evaluation.append(False)
    else:
        evaluation.append(True)

In [22]:
y_pred_df = pandas.DataFrame(list(zip(final_evaluation, evaluation, y_real,
                                       y_hat)), columns=['loss_dif',
                                                         'evaluation',
                                                         'y_real',
                                                         'y_hat'])

In [23]:
errors = y_pred_df.loc[y_pred_df['evaluation'] == False]

In [24]:
print(classification_report(
    y_pred_df.y_real.astype(numpy.float16).to_numpy(),
    numpy.where(y_pred_df.y_hat.astype(numpy.float16).to_numpy() > 0.5, 1, 0)
))

              precision    recall  f1-score   support

         0.0       0.67      0.83      0.74        42
         1.0       0.90      0.80      0.85        83

    accuracy                           0.81       125
   macro avg       0.79      0.81      0.80       125
weighted avg       0.83      0.81      0.81       125



In [25]:
y_pred = model.predict(X)

In [26]:
final_evaluation = []
y_pred_hat = []
y_real = []
evaluation = []

In [27]:
y_hat = [Y.to_numpy(), numpy.zeros(Y.to_numpy().shape[0], dtype=numpy.int16)]

In [28]:
for i in range(Y.to_numpy().shape[0]):
    if y_hat[0][i] == 0:
        y_hat[1][i] = 1
    else:
        y_hat[1][i] = 0

In [29]:
y_hat = numpy.transpose(y_hat)

In [30]:
y_hat = y_hat[:,[0, 1]] = y_hat[:,[1, 0]]

In [31]:
for i in range(Y.to_numpy().shape[0]):
    final_evaluation.append(y_hat[i][0] - y_pred[i][0])
    y_real.append(y_hat[i][0])
    y_pred_hat.append(y_pred[i][0])
    if numpy.mean(numpy.abs(y_hat[i] - y_pred[i])) > 0.5:
        evaluation.append(False)
    else:
        evaluation.append(True)

In [32]:
y_pred_df = pandas.DataFrame(list(zip(final_evaluation, evaluation, y_real,
                                       y_pred_hat)), columns=['loss_dif',
                                                         'evaluation',
                                                         'y_real',
                                                         'y_hat'])

In [33]:
errors = y_pred_df.loc[y_pred_df['evaluation'] == False]

In [34]:
print(classification_report(
    y_pred_df.y_real.astype(numpy.float16).to_numpy(),
    numpy.where(y_pred_df.y_hat.astype(numpy.float16).to_numpy() > 0.5, 1, 0)
))

              precision    recall  f1-score   support

         0.0       0.88      0.94      0.91       187
         1.0       0.96      0.92      0.94       313

    accuracy                           0.93       500
   macro avg       0.92      0.93      0.93       500
weighted avg       0.93      0.93      0.93       500

